In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.offline as plot
import json
print(pd.__version__)

2.2.2


## Export informace text as json

In [47]:
df = pd.read_json('data/pcr_udeska.json')

# Drop rows where the "informace" field is empty
df = df[df['informace'].astype(bool)]

# Extract the required fields and filter by "Oznámení o nálezu"
filtered_info = []
for entry in df['informace']:
    for item in entry:
        if 'název' in item and 'vyvěšení' in item:
            cs_content = item['název'].get('cs', '')
            vyeveseni_datum = item['vyvěšení'].get('datum', '')
            if "Oznámení o nálezu" in cs_content:
                filtered_info.append({
                    'vyvěšení': vyeveseni_datum,
                    'název': cs_content
                })

filtered_df = pd.DataFrame(filtered_info)

#filtered_df.to_json('data/pcr_filtered_informace.json', index=False, force_ascii=False, orient='records')


In [26]:

# Load the GeoJSON data
geodf = gpd.read_file("data/mapy/czech-regions-low-res.json")
geojson = json.loads(geodf.to_json())

# Create a DataFrame with the required columns
data = pd.DataFrame({
    'region': geodf['name'],  
    'value': geodf['population']  
})

# Create the Choropleth map
fig = px.choropleth(data, 
                    geojson=geojson, 
                    locations='region', 
                    featureidkey="properties.name",  # Adjust this key to match your GeoJSON properties
                    color='value',
                    projection="mercator"
                    )

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()


ValueError: 
    Invalid value of type 'builtins.str' received for the 'fitbounds' property of layout.geo
        Received value: 'region'

    The 'fitbounds' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            [False, 'locations', 'geojson']